In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from datetime import datetime

# --- CONFIG ---
MODEL_PATH = "models/saved_model.h5"
LABELS = ["hello", "yes", "no", "thank_you", "please"]  # replace with your labels
INPUT_SIZE = (224, 224)
START_HOUR = 18
END_HOUR = 22

# --- LOAD MODEL ---
model = load_model(MODEL_PATH)

# --- UTILS ---
def is_within_hours():
    now = datetime.now().time()
    start = datetime.strptime(f"{START_HOUR}:00", "%H:%M").time()
    end = datetime.strptime(f"{END_HOUR}:00", "%H:%M").time()
    return start <= now <= end

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, INPUT_SIZE)
    img = img / 255.0
    return np.expand_dims(img, axis=0)

def predict_image(img_path):
    x = preprocess_image(img_path)
    pred = model.predict(x)[0]
    idx = int(np.argmax(pred))
    return LABELS[idx], pred[idx]

# --- GUI ---
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Detection")
        self.root.geometry("600x500")

        # Image Label
        self.img_label = tk.Label(root)
        self.img_label.pack(pady=10)

        # Buttons
        self.upload_btn = tk.Button(root, text="Upload Image", command=self.upload_image)
        self.upload_btn.pack(pady=10)

        self.webcam_btn = tk.Button(root, text="Start Webcam", command=self.start_webcam)
        self.webcam_btn.pack(pady=10)

        # Prediction Label
        self.pred_label = tk.Label(root, text="", font=("Arial", 18))
        self.pred_label.pack(pady=20)

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if not file_path:
            return
        img = Image.open(file_path)
        img.thumbnail((300, 300))
        imgtk = ImageTk.PhotoImage(img)
        self.img_label.configure(image=imgtk)
        self.img_label.image = imgtk

        label, confidence = predict_image(file_path)
        self.pred_label.config(text=f"Prediction: {label} ({confidence:.2f})")

    def start_webcam(self):
        if not is_within_hours():
            messagebox.showwarning("Time Restriction", "Webcam prediction is allowed only from 6 PM to 10 PM!")
            return

        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            messagebox.showerror("Error", "Cannot open webcam")
            return

        cv2.namedWindow("Press 'q' to quit")

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = cv2.resize(frame_rgb, INPUT_SIZE) / 255.0
            pred = model.predict(np.expand_dims(img, axis=0))[0]
            idx = int(np.argmax(pred))
            label = LABELS[idx]

            cv2.putText(frame, f"{label} ({pred[idx]:.2f})", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Press 'q' to quit", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


# --- RUN APP ---
if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
